# Network Meta-Analysis (NMA) Validation Notebook

This notebook validates the Network Meta-Analysis VOI implementation in voiage by comparing results with theoretical expectations and demonstrating the functionality.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from voiage.methods.network_nma import (
    evsi_nma,
    simulate_nma_network_data,
    _perform_network_meta_analysis
)
from voiage.schema import ParameterSet, ValueArray, TrialDesign, DecisionOption

## Simple NMA Model

Let's create a simple NMA model with three treatments: Standard Care, New Treatment A, and New Treatment B.

In [ ]:
def simple_nma_economic_model(parameter_samples, trial_design=None, trial_data=None):
    """Simple NMA economic model that evaluates treatment strategies.
    
    Args:
        parameter_samples: ParameterSet with NMA parameters
        trial_design: Optional trial design
        trial_data: Optional trial data
        
    Returns:
        ValueArray: Net benefit values for each strategy
    """
    # Extract parameters
    if isinstance(parameter_samples, ParameterSet):
        params = parameter_samples.parameters
    else:
        params = parameter_samples
    
    # Get treatment effects (relative to standard care)
    if 'te_treatment_a' in params:
        te_a = params['te_treatment_a']
    else:
        te_a = np.full_like(next(iter(params.values())), 0.1)
        
    if 'te_treatment_b' in params:
        te_b = params['te_treatment_b']
    else:
        te_b = np.full_like(next(iter(params.values())), 0.2)
    
    # Baseline effectiveness (standard care)
    if 'baseline_effectiveness' in params:
        baseline = params['baseline_effectiveness']
    else:
        baseline = np.full_like(next(iter(params.values())), 0.6)
    
    # Calculate absolute effectiveness for each treatment
    effectiveness_standard = baseline
    effectiveness_a = baseline + te_a
    effectiveness_b = baseline + te_b
    
    # Cost parameters (£)
    if 'cost_standard' in params:
        cost_standard = params['cost_standard']
    else:
        cost_standard = np.full_like(next(iter(params.values())), 2000)
        
    if 'cost_a' in params:
        cost_a = params['cost_a']
    else:
        cost_a = np.full_like(next(iter(params.values())), 3000)
        
    if 'cost_b' in params:
        cost_b = params['cost_b']
    else:
        cost_b = np.full_like(next(iter(params.values())), 3500)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_threshold = 20000
    
    # Calculate net benefits for each strategy
    n_samples = len(effectiveness_standard)
    n_strategies = 3
    
    net_benefits = np.zeros((n_samples, n_strategies))
    
    # Standard care net benefit
    net_benefits[:, 0] = effectiveness_standard * wtp_threshold - cost_standard
    # Treatment A net benefit
    net_benefits[:, 1] = effectiveness_a * wtp_threshold - cost_a
    # Treatment B net benefit
    net_benefits[:, 2] = effectiveness_b * wtp_threshold - cost_b
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "strategy": ("n_strategies", ["Standard Care", "Treatment A", "Treatment B"])
        }
    )
    
    return ValueArray(dataset=dataset)

## Generate Parameter Samples

Let's generate parameter samples for our NMA probabilistic sensitivity analysis (PSA).

In [ ]:
# Generate parameter samples
np.random.seed(42)  # For reproducibility
n_samples = 1000

# Treatment effects (relative to standard care)
te_a = np.random.normal(0.1, 0.05, n_samples)  # Mean 0.1 QALYs gain
te_b = np.random.normal(0.2, 0.05, n_samples)  # Mean 0.2 QALYs gain

# Baseline effectiveness
baseline_effectiveness = np.random.normal(0.6, 0.1, n_samples)  # Mean 0.6 QALYs

# Cost parameters (£)
cost_standard = np.random.normal(2000, 200, n_samples)    # Mean £2000
cost_a = np.random.normal(3000, 300, n_samples)           # Mean £3000
cost_b = np.random.normal(3500, 350, n_samples)           # Mean £3500

# Create ParameterSet
parameters = {
    'te_treatment_a': te_a,
    'te_treatment_b': te_b,
    'baseline_effectiveness': baseline_effectiveness,
    'cost_standard': cost_standard,
    'cost_a': cost_a,
    'cost_b': cost_b
}

parameter_set = ParameterSet.from_numpy_or_dict(parameters)
print(f"Generated {parameter_set.n_samples} parameter samples")

## Calculate Net Benefits

Now let's calculate the net benefits for our treatment strategies.

In [ ]:
# Calculate net benefits
net_benefit_array = simple_nma_economic_model(parameter_set)
print(f"Net benefit array shape: {net_benefit_array.values.shape}")

# Show mean net benefits for each strategy
mean_nb = np.mean(net_benefit_array.values, axis=0)
strategies = net_benefit_array.strategy_names
for i, (strategy, nb) in enumerate(zip(strategies, mean_nb)):
    print(f"Mean net benefit for {strategy}: £{nb:,.0f}")

## Define a New Trial for NMA

Let's define a new trial that would provide additional evidence for our NMA.

In [ ]:
# Define a new trial comparing Treatment A vs Treatment B
trial_arms = [
    DecisionOption(name="Treatment A", sample_size=100),
    DecisionOption(name="Treatment B", sample_size=100)
]
new_trial_design = TrialDesign(arms=trial_arms)

print(f"New trial design: {new_trial_design.arms}")
print(f"Total sample size: {new_trial_design.total_sample_size}")

## Calculate EVSI-NMA

Now let's calculate the Expected Value of Sample Information for our proposed NMA study.

In [ ]:
# Calculate EVSI-NMA
evsi_nma_value = evsi_nma(
    nma_model_evaluator=simple_nma_economic_model,
    psa_prior_nma=parameter_set,
    trial_design_new_study=new_trial_design,
    n_outer_loops=10,  # Reduced for faster computation
    n_inner_loops=50
)

print(f"EVSI-NMA: £{evsi_nma_value:,.0f}")

## Population-Adjusted EVSI-NMA

Let's calculate the population-adjusted EVSI-NMA for a larger population.

In [ ]:
# Calculate population-adjusted EVSI-NMA
evsi_nma_pop = evsi_nma(
    nma_model_evaluator=simple_nma_economic_model,
    psa_prior_nma=parameter_set,
    trial_design_new_study=new_trial_design,
    population=100000,  # 100,000 patients
    time_horizon=10,    # 10 years
    discount_rate=0.03, # 3% discount rate
    n_outer_loops=10,
    n_inner_loops=50
)

print(f"Population-adjusted EVSI-NMA: £{evsi_nma_pop:,.0f}")

## Simulate NMA Network Data

Let's demonstrate the NMA data simulation functionality.

In [ ]:
# Simulate data for a network with 4 treatments and 8 studies
treatment_effects, se_effects, study_designs = simulate_nma_network_data(
    n_treatments=4,
    n_studies=8,
    baseline_effect=0.1,
    heterogeneity=0.05
)

print("Simulated NMA Network Data:")
print(f"Number of treatments: 4")
print(f"Number of studies: {len(treatment_effects)}")
print(f"Treatment effects: {treatment_effects}")
print(f"Standard errors: {se_effects}")
print(f"Study designs: {study_designs}")

## Perform Network Meta-Analysis

Let's demonstrate the NMA functionality.

In [ ]:
# Perform a simple NMA
nma_results, nma_variances = _perform_network_meta_analysis(
    treatment_effects,
    se_effects,
    study_designs
)

print("NMA Results:")
print(f"Treatment effects: {nma_results}")
print(f"Treatment variances: {nma_variances}")

## Sensitivity Analysis

Let's perform a sensitivity analysis by varying the sample size of the proposed trial.

In [ ]:
# Vary sample size and calculate EVSI-NMA
sample_sizes = [50, 100, 200, 500]
evsi_values = []

for sample_size in sample_sizes:
    trial_arms = [
        DecisionOption(name="Treatment A", sample_size=sample_size),
        DecisionOption(name="Treatment B", sample_size=sample_size)
    ]
    trial_design = TrialDesign(arms=trial_arms)
    
    evsi_val = evsi_nma(
        nma_model_evaluator=simple_nma_economic_model,
        psa_prior_nma=parameter_set,
        trial_design_new_study=trial_design,
        n_outer_loops=5,
        n_inner_loops=20
    )
    
    evsi_values.append(evsi_val)
    print(f"Sample size: {sample_size*2} (total), EVSI-NMA: £{evsi_val:,.0f}")

## Visualization

Let's visualize how EVSI-NMA changes with sample size.

In [ ]:
# Plot EVSI-NMA vs Sample Size
plt.figure(figsize=(10, 6))
plt.plot([size*2 for size in sample_sizes], evsi_values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Total Sample Size')
plt.ylabel('EVSI-NMA (£)')
plt.title('EVSI-NMA vs Sample Size for NMA Study')
plt.grid(True, alpha=0.3)
plt.show()

## Summary

This notebook has demonstrated:

1. How to set up an NMA model with multiple treatment options
2. How to generate parameter samples for probabilistic sensitivity analysis
3. How to calculate EVSI-NMA for a proposed study
4. How to perform population-adjusted calculations
5. How to simulate NMA network data
6. How to perform sensitivity analysis on sample size

The voiage NMA implementation provides a flexible framework for evaluating the value of proposed network meta-analysis studies.